Reproduce Elis' work, for my understanding

It looks like the three relevant bits of code happen in three steps:
- Define the uNet
- straighten the images
- perform segmentation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""
Define the uNet

"""
from cementum import unet

model = unet.my_unet()

In [ ]:
"""
Resize the images for training

"""
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from cementum import processing

train_size = (320, 320)

train_paths = tuple(pathlib.Path("../data/images").glob(r"?.jpg"))
full_size_images = [Image.open(str(path)) for path in train_paths]
training_images = [processing.resize(image, train_size) for image in full_size_images]

mask_paths = tuple(pathlib.Path("../data/masks").glob(r"?.jpg"))
training_masks = [
    processing.resize(Image.open(str(path)), train_size) for path in mask_paths
]

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(training_images[0])
axes[1].imshow(training_masks[0])

for axis, title in zip(axes, ("Image", "Mask")):
    axis.axis("off")
    axis.set_title(title)

In [ ]:
"""
Encode the labels and add this label to the data in the model; also compute class weights

"""

In [ ]:
"""
Train the model

"""

In [ ]:
"""
Plot the loss and validation accuracy at each epoch

"""

In [ ]:
"""
Calculate intersection over union for each class

"""

In [ ]:
"""
Apply to one of the large images

"""